In [2]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 17.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 13.3 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [3]:
from datasets import load_dataset
dataset = load_dataset("imagefolder", data_dir="dataset location")

Resolving data files:   0%|          | 0/4217 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [4]:
labels = dataset["train"].features["label"].names
label2id, id2label = dict(), dict()
for i, label in enumerate(labels):
    label2id[label] = i
    id2label[i] = label

id2label[2]

'glaucoma'

In [5]:
# Load model directly
from transformers import AutoImageProcessor, AutoModelForImageClassification

processor = AutoImageProcessor.from_pretrained("microsoft/resnet-50")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/266 [00:00<?, ?B/s]

In [14]:
processor

ConvNextImageProcessor {
  "crop_pct": 0.875,
  "do_normalize": true,
  "do_rescale": true,
  "do_resize": true,
  "image_mean": [
    0.485,
    0.456,
    0.406
  ],
  "image_processor_type": "ConvNextImageProcessor",
  "image_std": [
    0.229,
    0.224,
    0.225
  ],
  "resample": 3,
  "rescale_factor": 0.00392156862745098,
  "size": {
    "shortest_edge": 224
  }
}

In [6]:
def preprocess_images(example):
    inputs = processor(example['image'], return_tensors="pt")
    example['pixel_values'] = inputs['pixel_values'].squeeze()  # Remove extra batch dimension
    return example

In [7]:
processed_dataset = dataset.map(preprocess_images)

Map:   0%|          | 0/4217 [00:00<?, ? examples/s]

In [8]:
splits = processed_dataset["train"].train_test_split(test_size=0.3)
val_test_split = splits['test'].train_test_split(test_size=0.5)  # 15% validation, 15% test

train_ds = splits['train']
val_ds = val_test_split['train']
test_ds = val_test_split['test']

In [9]:
train_ds

Dataset({
    features: ['image', 'label', 'pixel_values'],
    num_rows: 2951
})

In [10]:
val_ds

Dataset({
    features: ['image', 'label', 'pixel_values'],
    num_rows: 633
})

In [11]:
test_ds

Dataset({
    features: ['image', 'label', 'pixel_values'],
    num_rows: 633
})

In [12]:
label2id

{'cataract': 0, 'diabetic_retinopathy': 1, 'glaucoma': 2, 'normal': 3}

In [13]:
id2label

{0: 'cataract', 1: 'diabetic_retinopathy', 2: 'glaucoma', 3: 'normal'}

In [15]:
model = AutoModelForImageClassification.from_pretrained(
    "microsoft/resnet-50",
    label2id=label2id,
    id2label=id2label,
    ignore_mismatched_sizes=True,
)

config.json:   0%|          | 0.00/69.6k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/102M [00:00<?, ?B/s]

Some weights of ResNetForImageClassification were not initialized from the model checkpoint at microsoft/resnet-50 and are newly initialized because the shapes did not match:
- classifier.1.bias: found shape torch.Size([1000]) in the checkpoint and torch.Size([4]) in the model instantiated
- classifier.1.weight: found shape torch.Size([1000, 2048]) in the checkpoint and torch.Size([4, 2048]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [16]:
from transformers import TrainingArguments, Trainer
args = TrainingArguments(
    output_dir="/content/drive/MyDrive/Mediscan'/Resnet",
    per_device_train_batch_size=16,
    eval_strategy="steps",
    num_train_epochs=4,
    fp16=True,
    save_steps=200,
    eval_steps=200,
    logging_steps=100,
    learning_rate=2e-4,
    save_total_limit=2,
    remove_unused_columns=False,
    push_to_hub=False,
    report_to='tensorboard',
    load_best_model_at_end=True,
)


In [17]:
!pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 3.7 MB/s eta 0:00:00


In [18]:
import torch
import numpy as np

from evaluate import load
# Collate function for creating batches
def collate_fn(examples):
    pixel_values = torch.stack([torch.tensor(example["pixel_values"]) for example in examples])
    labels = torch.tensor([example["label"] for example in examples])
    return {"pixel_values": pixel_values, "labels": labels}

# Define the accuracy metric
metric = load("accuracy")
def compute_metrics(eval_pred):
    """Computes accuracy on a batch of predictions."""
    predictions = np.argmax(eval_pred.predictions, axis=1)
    return metric.compute(predictions=predictions, references=eval_pred.label_ids)


In [19]:
trainer = Trainer(
    model,
    args,
    train_dataset=train_ds,
    eval_dataset=val_ds,
    compute_metrics=compute_metrics,
    data_collator=collate_fn,
)

In [20]:
train_results = trainer.train()
trainer.save_model()
trainer.log_metrics("train", train_results.metrics)
trainer.save_metrics("train", train_results.metrics)
trainer.save_state()
print("Train Results:", train_results)


Step,Training Loss,Validation Loss,Accuracy
200,0.649900,0.465681,0.824645
400,0.331400,0.324213,0.879937
600,0.241000,0.307160,0.881517


***** train metrics *****
  epoch                    =         4.0
  total_flos               = 233488521GF
  train_loss               =      0.4481
  train_runtime            =  0:25:40.54
  train_samples_per_second =       7.662
  train_steps_per_second   =        0.48
Train Results: TrainOutput(global_step=740, training_loss=0.4480928234151892, metrics={'train_runtime': 1540.547, 'train_samples_per_second': 7.662, 'train_steps_per_second': 0.48, 'total_flos': 2.507063908200284e+17, 'train_loss': 0.4480928234151892, 'epoch': 4.0})


In [21]:
metrics = trainer.evaluate()
trainer.log_metrics("eval", metrics)
trainer.save_metrics("eval", metrics)
print("Evaluate Results:", metrics)


***** eval metrics *****
  epoch                   =        4.0
  eval_accuracy           =     0.8815
  eval_loss               =     0.3072
  eval_runtime            = 0:01:08.18
  eval_samples_per_second =      9.284
  eval_steps_per_second   =      1.173
Evaluate Results: {'eval_loss': 0.307160347700119, 'eval_accuracy': 0.8815165876777251, 'eval_runtime': 68.1818, 'eval_samples_per_second': 9.284, 'eval_steps_per_second': 1.173, 'epoch': 4.0}


In [22]:
test_results = trainer.evaluate(test_ds)
print("Test Results:", test_results)

Test Results: {'eval_loss': 0.22871015965938568, 'eval_accuracy': 0.9225908372827805, 'eval_runtime': 68.2959, 'eval_samples_per_second': 9.268, 'eval_steps_per_second': 1.171, 'epoch': 4.0}
